In [1]:

import numpy as np
import pickle
import os
from tqdm import tqdm

import pandas as pd
import json

import re
import numpy as np
import tensorflow as tf
from tensorflow import keras

2025-12-04 13:24:28.466295: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-04 13:24:28.466542: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 13:24:28.517075: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 13:24:29.717693: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
numeric_pattern = r"\(\s*\d+\s*/\s*\d+\s*\)"
english = r"[a-zA-Z]"
numbers = r"\s*\d+\s*"
numering_items = r"\s*\d+\s*[-]\s*"
empty_brackets = r'\(\s*\)|\[\s*\]|\{\s*\}|<<\s*>>|"\s*"|\'\s*\''
stand_alone=r'(?<=\s|\^|\(|\[|\{)[^\(\)\[\]\{\}\.,،:;؛؟!\-](?=\s|$|\]|\)|\})'
UNK_CHAR = '?'

In [3]:
kawloho_pattern = r"(\s*قَوْلُهُ\s*)"
qala_variations = r"(?:قَالَ|قَالَتْ|قُلْت|قَالُوا|قُلْنَا|أَقُولُ)"
qala_variations = r"(?:قَالَ|قَالَتْ|قُلْت|قَالُوا|قُلْنَا|أَقُولُ)"
qala_pattern = rf"(\s*{qala_variations}\s*:)"
INTAHA = r'\s+ا\s*هـ?\s+'
DIACRITICS_PATTERN = re.compile(r'[\u064B-\u0652]')

In [4]:
with open('/home/zizo/Documents/NLP/Project/data/val.txt') as f:
    lines = f.readlines()
with open('/home/zizo/Documents/NLP/Project/utils/diacritics.pickle', 'rb') as f:
    diacritic2id = pickle.load(f)

In [5]:
diacritic2id = {
    "َ": 0,
    "ً": 1,
    "ُ": 2,
    "ٌ": 3,
    "ِ": 4,
    "ٍ": 5,
    "ْ": 6,
    "ّ": 7,
    "َّ": 8,
    "ًّ": 9,
    "ُّ": 10,
    "ٌّ": 11,
    "ِّ": 12,
    "ٍّ": 13,
    "": 14
}

In [6]:
def remove_unbalanced_brackets(text):
    pair_map = {')': '(', '}': '{', ']': '[', '>':'<', '»': '«', '"':'"', "'":"'"}
    openers = set(['(', '{', '[', '<', '«', '"', "'"])
    
    stack = [] 
    indices_to_remove = set()

    for i, char in enumerate(text):
        if char in openers:
            stack.append((char, i))
        
        elif char in pair_map:
            if stack:
                last_opener, _ = stack[-1]
                if last_opener == pair_map[char]:
                    stack.pop()
                else:
                    indices_to_remove.add(i)
            else:
                indices_to_remove.add(i)

    for char, index in stack:
        indices_to_remove.add(index)

    return "".join([char for i, char in enumerate(text) if i not in indices_to_remove])


In [7]:
def clean_punctuation_sequence(text):
    collapsible = re.escape(".,:;!?'\"/،؛؟")
    pattern = rf"([{collapsible}])(?:\s*\1)+"

    return re.sub(pattern, r"\1", text)

In [8]:
def split_citations_raw(line):
    qal_list = [
        "قال", "قالت", "قالوا", "قلت", "قلنا",
        "أقول", "يقول", "يقولون", "قيل", "يقال"
    ]

    qal_regex = "|".join(qal_list)

    qal_with_colon = rf"(?:{qal_regex})\s*[:：]"

    
    qawloho_regex = r"(?:و|ف)?قول(?:ه)?(?:\s*تعالى)?"

    trigger = rf"({qal_with_colon}|{qawloho_regex})"

    final_lines = []
    matches = list(re.finditer(trigger, line))
    
    if not matches:
        final_lines.append(line.strip())
    else:
        last_idx = 0
        for m in matches:
            start = m.start()
            if line[last_idx:start]:
                final_lines.append(line[last_idx:start])
            last_idx = start
        
        final_lines.append(line[last_idx:])

    return final_lines

In [9]:
def initial_process(line):
    res = re.sub(numering_items, '', line)
    res = re.sub(numeric_pattern, '', res)
    res = re.sub(english, ' ', res)
    res = re.sub(numbers, '', res)
    res = re.sub(empty_brackets, '', res)
    res = re.sub(',', '،', res)
    res = re.sub(';', '؛', res)
    res = re.sub(r'\?', '؟', res)
    res = re.sub(r'/', '', res)
    res = re.sub(r'\*', '', res)
    res = re.sub(r'–', '-', res)
    res = res.replace('\u200f', '')
    

    res = clean_punctuation_sequence(res)

    res = remove_unbalanced_brackets(res)

    res = re.sub(r"\s+", " ", res).strip()

    return res

In [10]:
def slide_window_raw(text, overlap=50, max_len=200):
    if len(text) <= max_len:
        return [text], []
    
    chunks = []
    overlaps = []
    
    chunks.append(text[:max_len])
    
    current_start = 0
    text_len = len(text)
    
    while True:
        ideal_stride = max_len - overlap
        
        ideal_next_start = current_start + ideal_stride
        
        if ideal_next_start >= text_len:
            break

        found_next_start = -1
        
        search_limit = current_start  
        
        for i in range(ideal_next_start, search_limit, -1):
            if i < text_len and text[i] == ' ':
                found_next_start = i + 1 
                break
        
        if found_next_start == -1:
            found_next_start = ideal_next_start
            
        
        actual_overlap = (current_start + max_len) - found_next_start
        
        if actual_overlap < 0:
            actual_overlap = 0

        next_chunk = text[found_next_start : found_next_start + max_len]
        
        chunks.append(next_chunk)
        overlaps.append(actual_overlap)
        
        current_start = found_next_start
        
        if current_start + max_len >= text_len:
            break
            
    return chunks, overlaps

In [11]:
def reconstruct_text_window(chunks, overlaps):
    if not chunks:
        return ""
    
    reconstructed_parts = [chunks[0]]
    
    for chunk, ov in zip(chunks[1:], overlaps):
        reconstructed_parts.append(chunk[ov:])
        
    return "".join(reconstructed_parts)

In [12]:
def split_text_and_diacritics(text):

    letters = []
    labels = []

    i = 0
    while i < len(text):
        char = text[i]

        if DIACRITICS_PATTERN.match(char):
            if labels:
                labels[-1] += char
        else:
            letters.append(char)
            labels.append("")

        i += 1

    return "".join(letters), labels

In [13]:
# def predict(cleaned_lines):
    
#     test_x = []
#     for sent in tqdm(cleaned_lines):
#         sentence_vec = zizo_features(sent)
#         test_x.append(sentence_vec)

#     def test_generator():
#         for features in test_x:
#             yield features

#     test_dataset = tf.data.Dataset.from_generator(
#         test_generator,
#         output_signature=tf.TensorSpec(shape=(None, INPUT_DIM), dtype=tf.float32)
#     )

#     test_dataset = test_dataset.padded_batch(
#         32,
#         padding_values=PADDING_INPUT 
#     )
        
#     predicted_tags = []
    
#     for batch_x in tqdm(test_dataset): 
#         batch_probs = model.predict_on_batch(batch_x)
#         batch_pred_ids = np.argmax(batch_probs, axis=-1)
        
#         batch_size = batch_pred_ids.shape[0]
        
#         for k in range(batch_size):
            
#             valid_mask = np.any(batch_x[k] != PADDING_INPUT, axis=-1) 
           
#             real_len = np.sum(valid_mask)
            
#             p_seq = batch_pred_ids[k][:real_len]
            
#             predicted_tags.append(p_seq)

#     return predicted_tags

In [14]:
def prepare_for_predict():
    all_text_chunks = []
    all_recovery = []
    all_overlaps = [] 
    length = 0
    assertions = []
    
    with open('/home/zizo/Documents/NLP/Project/data/val.txt', "r", encoding="utf-8") as file:
        
        for line in file:
            length += 1

            line, _ = split_text_and_diacritics(line)
            recovery = []
            curr_chunks = []
            curr_overlaps = [] 
            
            cleaned = initial_process(line.strip())
            assertions.append(cleaned)
            
            raw_segments = split_citations_raw(cleaned)

            for seg in raw_segments:
                t_chunks, t_overlaps = slide_window_raw(seg, overlap=50, max_len=807)
                
                for i, chunk in enumerate(t_chunks):
                    recovery.append(i)
                    curr_chunks.append(chunk)
                
               
                curr_overlaps.extend(t_overlaps)

            all_recovery.append(recovery)
            all_text_chunks.append(curr_chunks)
            all_overlaps.append(curr_overlaps) 
            
    print(f"Generated {len(all_text_chunks)} chunks.")
    return all_text_chunks, all_overlaps, all_recovery, length, assertions

In [15]:
all_text_chunks, all_overlaps, all_recovery, length, assertions = prepare_for_predict()

Generated 2500 chunks.


In [16]:
reconstructed = []

for i in range(len(all_recovery)):
    res = ''
    curr_chunks = []
    curr_overlaps = []
    
    overlap_idx = 0 
    
    for j in range(len(all_recovery[i])):
        recovery_id = all_recovery[i][j]
        chunk = all_text_chunks[i][j]
        
        if recovery_id == 0:
            if curr_chunks:
                res += reconstruct_text_window(curr_chunks, curr_overlaps)
            
            curr_chunks = [chunk]
            curr_overlaps = [] 
        
        else:
            curr_chunks.append(chunk)
            
            if overlap_idx < len(all_overlaps[i]):
                curr_overlaps.append(all_overlaps[i][overlap_idx])
                overlap_idx += 1

    if curr_chunks:
        res += reconstruct_text_window(curr_chunks, curr_overlaps)
        
    reconstructed.append(res)

In [17]:
with open('/home/zizo/Documents/NLP/Project/utils/arabic_letters.pickle', "rb") as file:
    ARABIC_CHARS = pickle.load(file)

In [18]:
def arabic_only(text):
    return "".join([char for char in text if char in ARABIC_CHARS or char == " "])

In [19]:
for i in range(len(reconstructed)):
    reconstructed[i] = arabic_only(reconstructed[i])
    assertions[i] = arabic_only(assertions[i])
    reconstructed[i] = re.sub(r"\s+", " ", reconstructed[i])
    assertions[i] = re.sub(r"\s+", " ", assertions[i])

In [20]:
assert len(reconstructed) == length
for i in range(len(reconstructed)):
    assert len(reconstructed[i].strip()) == len(assertions[i].strip()), print(reconstructed[i], "\n", assertions[i])